In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import OneHotEncoderEstimator, MinMaxScaler, StringIndexer, VectorAssembler
from pyspark.ml import Pipeline, PipelineModel
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [2]:
conf = SparkConf().setMaster('local').setAppName('Predict Adult Salary')
sc = SparkContext(conf = conf)

In [3]:
spark = SparkSession \
    .builder \
    .appName("Predict Adult Salary") \
    .getOrCreate()

In [4]:
schema = StructType([
    StructField("age", IntegerType(), True),
    StructField("workclass", StringType(), True),
    StructField("fnlwgt", IntegerType(), True),
    StructField("education", StringType(), True),
    StructField("education-num", IntegerType(), True),
    StructField("marital-status", StringType(), True),
    StructField("occupation", StringType(), True),
    StructField("relationship", StringType(), True),
    StructField("race", StringType(), True),
    StructField("sex", StringType(), True),
    StructField("capital-gain", IntegerType(), True),
    StructField("capital-loss", IntegerType(), True),
    StructField("hours-per-week", IntegerType(), True),
    StructField("native-country", StringType(), True),
    StructField("salary", StringType(), True)
])

In [5]:
# train_df = spark.read.format("csv").load('application_train.csv')
train_df = spark.read.csv('train.csv', header=False, schema=schema)

test_df = spark.read.csv('test.csv', header=False, schema=schema)

In [6]:
train_df.head(5)

[Row(age=39, workclass='State-gov', fnlwgt=77516, education='Bachelors', education-num=13, marital-status='Never-married', occupation='Adm-clerical', relationship='Not-in-family', race='White', sex='Male', capital-gain=2174, capital-loss=0, hours-per-week=40, native-country='United-States', salary='<=50K'),
 Row(age=50, workclass='Self-emp-not-inc', fnlwgt=83311, education='Bachelors', education-num=13, marital-status='Married-civ-spouse', occupation='Exec-managerial', relationship='Husband', race='White', sex='Male', capital-gain=0, capital-loss=0, hours-per-week=13, native-country='United-States', salary='<=50K'),
 Row(age=38, workclass='Private', fnlwgt=215646, education='HS-grad', education-num=9, marital-status='Divorced', occupation='Handlers-cleaners', relationship='Not-in-family', race='White', sex='Male', capital-gain=0, capital-loss=0, hours-per-week=40, native-country='United-States', salary='<=50K'),
 Row(age=53, workclass='Private', fnlwgt=234721, education='11th', educati

In [8]:
train_df.limit(5).toPandas()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [ ]:
# indexer = StringIndexer(inputCols=['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country'], outputCols=['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country'])
# model = indexer.fit(train_df)
# indexed = indexer.transform(train_df)

# encoder = OneHotEncoderEstimator(inputCols=['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country'], outputCols=['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country'])
# model = encoder.fit(train_df)
# encoded = model.transform(train_df)

In [ ]:
# indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(train_df) for column in list(train_df.columns)]

# pipeline = Pipeline(stages=indexers)
# df_r = pipeline.fit(train_df).transform(train_df)

In [ ]:
# df_r.toPandas()

# indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(train_df) for column in list(train_df.columns)]

# indexers = StringIndexer(
#     inputCol = categorical_variables,
#     outputCol =  ['workclass_index', 'education_index', 'marital-status_index', 'occupation_index', 'relationship_index', 'race_index', 'sex_index', 'native-country_index']
# )  

In [19]:
categorical_variables = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']

indexers = [StringIndexer(inputCol=column, outputCol=column+"_index") for column in categorical_variables]

encoder = OneHotEncoderEstimator(
    inputCols=[indexer.getOutputCol() for indexer in indexers],
    outputCols=[
        "{0}_encoded".format(indexer.getOutputCol()) for indexer in indexers]
)

assembler = VectorAssembler(
    inputCols=encoder.getOutputCols(),
    outputCol="features"
)

scaler = MinMaxScaler(
    inputCol="features",
    outputCol="scaledFeatures"
)


pipeline = Pipeline(stages=indexers + [encoder, assembler, scaler])

transformed = pipeline.fit(train_df).transform(train_df)

In [20]:
transformed.printSchema()

root
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: integer (nullable = true)
 |-- education: string (nullable = true)
 |-- education-num: integer (nullable = true)
 |-- marital-status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- capital-gain: integer (nullable = true)
 |-- capital-loss: integer (nullable = true)
 |-- hours-per-week: integer (nullable = true)
 |-- native-country: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- workclass_index: double (nullable = false)
 |-- education_index: double (nullable = false)
 |-- marital-status_index: double (nullable = false)
 |-- occupation_index: double (nullable = false)
 |-- relationship_index: double (nullable = false)
 |-- race_index: double (nullable = false)
 |-- sex_index: double (nullable = false)
 |-- native-country_index: double 

In [ ]:
# scaler = MinMaxScaler(inputCol="features",\
#          outputCol="scaledFeatures")
# scaler_model = scaler.fit(transformed.select("features"))

# scaled_data = scaler_model.transform(transformed)

In [21]:
# scaled_data.limit(5).toPandas()
df = transformed.limit(5).toPandas()

In [22]:
df['scaledFeatures'][1]

DenseVector([0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

In [26]:
continuous_variables = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']

assembler = VectorAssembler(
    inputCols=['scaledFeatures', *continuous_variables],
    outputCol='inputs'
)

X = assembler.transform(transformed)

In [29]:
X.limit(5).toPandas()['inputs'][0]

SparseVector(100, {4: 1.0, 10: 1.0, 24: 1.0, 32: 1.0, 44: 1.0, 48: 1.0, 52: 1.0, 53: 1.0, 94: 39.0, 95: 77516.0, 96: 13.0, 97: 2174.0, 99: 40.0})

In [34]:
indexer = StringIndexer(inputCol='salary', outputCol='label')
X = indexer.fit(X).transform(X)

In [35]:
X.limit(10).toPandas()['label']

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
5    0.0
6    0.0
7    1.0
8    1.0
9    1.0
Name: label, dtype: float64

In [36]:
lr = LogisticRegression(featuresCol='inputs', labelCol='label')

lr.fit(X)

LogisticRegressionModel: uid = LogisticRegression_a80a494fb9db, numClasses = 2, numFeatures = 100

In [ ]:
pred = lr.evaluate(test_data)

In [ ]:
pred.predictions.toPandas()